# 🎤 RVC Voice Cloning System - Google Colab

This notebook provides GPU access for users without local GPUs.

**Features**:
- 💾 **Google Drive Integration**: Automatically save and load trained models
- 🚀 **GPU Acceleration**: Uses Tesla T4/P100
- 🧠 **Real Training**: Uses official RVC backend for high-quality results
- 🔄 **RVC-Python**: Robust inference engine

## Setup Instructions

1.  Run all cells in order
2.  Mount Google Drive when prompted
3.  Use the training and inference cells below

## 🔌 Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
import os

print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create serialization directory on Drive
DRIVE_RVC_DIR = "/content/drive/MyDrive/RVC_Models"
os.makedirs(DRIVE_RVC_DIR, exist_ok=True)
print(f"✅ Google Drive mounted. Models will be saved to: {DRIVE_RVC_DIR}")

## 📦 Step 2: Clone Repository and Install Dependencies

In [ ]:
import os
import subprocess

# ⚠️ REPLACE WITH YOUR GITHUB REPO URL ⚠️
REPO_URL = "https://github.com/bherulalmali/rvc-system.git"
REPO_DIR = "rvcStudioAG"

if not os.path.exists(REPO_DIR):
    print(f"Cloning repository from {REPO_URL}...")
    try:
        subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
        print("✅ Repository cloned successfully")
    except subprocess.CalledProcessError:
        print("❌ Failed to clone. Please check the REPO_URL above.")
else:
    print(f"Repository already exists at {REPO_DIR}")

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"Working directory: {os.getcwd()}")
    # Removed requirements.txt install to prevent crashes

## 🔄 Step 3: Load Saved Models from Drive

Syncs models from your Google Drive `RVC_Models` folder to the local workspace.

In [ ]:
import shutil

local_models_dir = "models"
os.makedirs(local_models_dir, exist_ok=True)

print("Syncing models from Drive...")
if os.path.exists(DRIVE_RVC_DIR):
    # Iterate over subdirectories in Drive RVC folder
    synced_count = 0
    for item in os.listdir(DRIVE_RVC_DIR):
        drive_path = os.path.join(DRIVE_RVC_DIR, item)
        if os.path.isdir(drive_path):
            local_path = os.path.join(local_models_dir, item)
            if not os.path.exists(local_path):
                shutil.copytree(drive_path, local_path)
                synced_count += 1
                print(f"Synced voice: {item}")
    
    if synced_count == 0:
        print("No new models found on Drive to sync.")
    else:
        print(f"✅ Synced {synced_count} models from Google Drive")
else:
    print("Drive directory not found (should be empty if first run)")

## 🎓 Step 4: Train a New Voice (Real RVC Backend)

1. Enter the name of the person/character.
2. Click the upload button to select your `.wav` files.
3. The system will process, train (50 epochs by default), and save the model to your Drive.

In [ ]:
import os
import shutil
import subprocess
import re
import glob
import sys
from pathlib import Path
from google.colab import files

# 1. Inputs
PERSON_NAME = "my_voice" # @param {type:"string"}
EPOCHS = 50 # @param {type:"integer"}

print(f"🎤 Voice Name: {PERSON_NAME}")
print(f"🔄 Epochs: {EPOCHS}")

# 2. Upload Audio
print("\n📂 Please upload your audio files (.wav)...")
uploaded = files.upload()
AUDIO_FILES = list(uploaded.keys())

if not AUDIO_FILES:
    print("⚠️ No files uploaded. Please rerun this cell and upload audio.")
else:
    print(f"🚀 Initializing Real RVC Training for: {PERSON_NAME}")
    
    # 1. Setup Official RVC Backend (STEALTH MODE - No Clone)
    RVC_BACKEND_DIR = "training_core"
    
    # FORCE CLEANUP
    if os.path.exists(RVC_BACKEND_DIR):
        if not os.path.exists(os.path.join(RVC_BACKEND_DIR, "infer")):
             print("⚠️ Detected broken backend from previous run. Deleting...")
             shutil.rmtree(RVC_BACKEND_DIR)
             
    if not os.path.exists(RVC_BACKEND_DIR):
        print("📥 Downloading core assets (Safe Mode)...")
        subprocess.run("wget https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/archive/refs/heads/main.zip -O rvc_core.zip", shell=True, check=True)
        subprocess.run("unzip -q rvc_core.zip", shell=True, check=True)
        subprocess.run(f"mv Retrieval-based-Voice-Conversion-WebUI-main {RVC_BACKEND_DIR}", shell=True, check=True)
        subprocess.run("rm rvc_core.zip", shell=True, check=True)
        
        for f in ["README.md", "README.en.md", "docs"]:
            path = os.path.join(RVC_BACKEND_DIR, f)
            if os.path.exists(path):
                if os.path.isdir(path):
                    shutil.rmtree(path)
                else:
                    os.remove(path)
    else:
        print("✅ Backend directory exists")
    
    print("📦 Installing Verified RVC Dependencies (SEQUENTIAL MODE)...")
    
    def run_pip(pkg_name, cmd_override=None):
        print(f"... Installing {pkg_name}")
        cmd = cmd_override if cmd_override else f"pip install --no-cache-dir {pkg_name}"
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if res.returncode != 0:
            print(f"❌ FAILED {pkg_name} install! Output:\n{res.stdout}\n{res.stderr}")
            return False
        return True

    subprocess.run("sudo apt-get install -y libsndfile1-dev swig > /dev/null 2>&1", shell=True, check=True)

    run_pip("ninja")
    run_pip('"numpy<2.0"')
    
    # UPDATE: Use Python 3.12 compatible versions for core Hydra/Omegaconf
    print("... Installing modern omegaconf/hydra (wheels)")
    run_pip("omegaconf==2.3.0")
    run_pip("hydra-core==1.3.2")
    run_pip("antlr4-python3-runtime==4.9.3") 
    run_pip("bitarray") 
    run_pip("sacrebleu")

    deps = [
        "librosa==0.9.1", 
        "faiss-cpu",
        "praat-parselmouth==0.4.3",
        "pyworld==0.3.4",
        "tensorboardX",
        "torchcrepe",
        "ffmpeg-python",
        "av",
        "scipy",
        "protobuf==3.20.0"
    ]

    for dep in deps:
        run_pip(dep)

    print("... Installing fairseq (wheel info override)")
    # Ignoring dependencies is crucial as fairseq asks for old omega/hydra
    if not run_pip("fairseq==0.12.2", "pip install --no-cache-dir --no-deps fairseq==0.12.2"):
         print("⚠️ Wheel failed. Trying source...")
         run_pip("fairseq", "pip install --no-cache-dir git+https://github.com/facebookresearch/fairseq.git")

    # ==========================================================================
    # 🐍 PYTHON 3.12 COMPATIBILITY PATCHER (FAIRSEQ + HYDRA)
    # ==========================================================================
    print("🛠️ Running Python 3.12 Compatibility Patcher...")
    
    def patch_file(target_file, replacement_pairs, name="Unknown"):
        if os.path.exists(target_file):
             with open(target_file, "r") as f:
                 content = f.read()
             
             new_content = content
             # Always ensure 'field' is imported
             if "from dataclasses import" in new_content and "field" not in new_content:
                 new_content = new_content.replace("from dataclasses import", "from dataclasses import field,")
             # Hydra uses full import sometimes, so we add a specific backup check
             if "import dataclasses" in new_content:
                  # we might need to add our own import
                  new_content = "from dataclasses import field\n" + new_content

             for old, new in replacement_pairs:
                 new_content = new_content.replace(old, new)
             
             # Generic Regex fallback for any leftovers
             # Converts: `var: Type = Type()` -> `var: Type = field(default_factory=Type)`
             pattern = r"(\s+)(\w+): ([\w\.]+) = ([\w\.]+)\(\\)"
             new_content = re.sub(pattern, r"\1\2: \3 = field(default_factory=\4)", new_content)

             if content != new_content:
                 with open(target_file, "w") as f:
                     f.write(new_content)
                 print(f"✅ Patched {name} successfully.")
             else:
                 print(f"ℹ️ {name} seems already patched or compliant.")
        else:
             print(f"⚠️ Could not find file for {name}: {target_file}")

    try:
        # 1. Find packages root (site-packages OR dist-packages)
        # Google Colab uses dist-packages for system installs, which pip often uses
        site_dirs = [p for p in sys.path if ("site-packages" in p or "dist-packages" in p) and os.path.isdir(p)]
        
        if not site_dirs:
            print(f"❌ Could not locate package directory! Sys.path: {sys.path}")
        else:
            print(f"... Scanning {len(site_dirs)} package directories for libraries to patch...")
            
            # Iterate ALL valid dirs to find where packages surely are
            for base_dir in site_dirs:
                # --- PATCH 1: FAIRSEQ ---
                fairseq_config = os.path.join(base_dir, "fairseq", "dataclass", "configs.py")
                if os.path.exists(fairseq_config):
                    print(f"... Found fairseq in {base_dir}")
                    patch_file(fairseq_config, [
                        ("common: CommonConfig = CommonConfig()", "common: CommonConfig = field(default_factory=CommonConfig)"),
                        ("dataset: DatasetConfig = DatasetConfig()", "dataset: DatasetConfig = field(default_factory=DatasetConfig)"),
                        ("= FairseqBMUFConfig()", "= field(default_factory=FairseqBMUFConfig)"),
                    ], name="Fairseq Configs")

                # --- PATCH 2: HYDRA-CORE (JobConf) ---
                hydra_conf_init = os.path.join(base_dir, "hydra", "conf", "__init__.py")
                if os.path.exists(hydra_conf_init):
                    print(f"... Found hydra in {base_dir}")
                    patch_file(hydra_conf_init, [
                        ("override_dirname: OverrideDirname = OverrideDirname()", "override_dirname: OverrideDirname = field(default_factory=OverrideDirname)"),
                    ], name="Hydra JobConf")

    except Exception as e:
        print(f"⚠️ Patching Exception: {e}")
    # ==========================================================================

    # 3. Trigger Training
    print("🧠 Starting Feature Extraction and Training...")
    
    cwd_backup = os.getcwd()
    
    # Define Absolute paths
    rvc_internal_dataset_dir = os.path.join(cwd_backup, RVC_BACKEND_DIR, "dataset")
    dataset_abs_path = os.path.join(rvc_internal_dataset_dir, PERSON_NAME)
    logs_abs_path = os.path.join(cwd_backup, RVC_BACKEND_DIR, "logs", PERSON_NAME)
    
    os.makedirs(dataset_abs_path, exist_ok=True)
    os.makedirs(logs_abs_path, exist_ok=True)
    
    print(f"... Moving audio files to {dataset_abs_path}")
    for audio_file in AUDIO_FILES:
        if os.path.exists(audio_file):
            shutil.copy(audio_file, os.path.join(dataset_abs_path, audio_file))

    os.chdir(RVC_BACKEND_DIR)
    
    # DEBUG: Check file existence
    print("🔍 Validating backend files...")
    target_script = "infer/modules/train/extract/extract_f0_print.py"
    if not os.path.exists(target_script):
        print(f"❌ CRITICAL: Script not found: {target_script}")
    
    try:
        def run_cmd(cmd, hide_output=False):
            print(f"Running: {cmd.split()[0]} ... (args hidden)")
            if hide_output:
                result = subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, text=True)
            else:
                result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
                
            if result.returncode != 0:
                print(f"❌ Command Failed!\nSTDERR: {result.stderr}")
                raise RuntimeError(f"Command failed: {cmd}")
            print("✅ Done.")
            return result
        
        print("--- 1. Preprocessing Dataset ---")
        cmd_preprocess = f"python infer/modules/train/preprocess.py '{dataset_abs_path}' 40000 2 '{logs_abs_path}' False 3.0"
        run_cmd(cmd_preprocess, hide_output=True)

        print("--- 2. Extracting Pitch (F0) ---")
        run_cmd(f"python infer/modules/train/extract/extract_f0_print.py '{logs_abs_path}' 2 rmvpe", hide_output=True)
        
        print("--- 3. Extracting Features ---")
        run_cmd(f"python infer/modules/train/extract_feature_print.py cuda 1 0 0 '{logs_abs_path}' v2", hide_output=True)
        
        print("--- 4. Training Model ---")
        # Reduced batch size to 1 just in case, but 4 is usually fine on T4
        cmd_train = f"python infer/modules/train/train.py -e {PERSON_NAME} -sr 40k -ov 0 -bs 4 -te {EPOCHS} -pg 0 -if 0 -l 0 -c 0 -sw 0 -v v2"
        run_cmd(cmd_train, hide_output=False)
        
        # 4. Export Model
        print("✅ Training finished. Exporting model...")
        weights_dir = "weights"
        pth_files = [f for f in os.listdir(weights_dir) if PERSON_NAME in f and ".pth" in f]
        if pth_files:
             latest_model = sorted(pth_files)[-1]
             target_model_path = os.path.join(cwd_backup, "models", f"{PERSON_NAME}.pth")
             shutil.copy(os.path.join(weights_dir, latest_model), target_model_path)
             print(f"🏆 Model saved locally to: {target_model_path}")
             
             drive_voice_dir = os.path.join(DRIVE_RVC_DIR, PERSON_NAME)
             if not os.path.exists(drive_voice_dir):
                 os.makedirs(drive_voice_dir)
             shutil.copy(target_model_path, os.path.join(drive_voice_dir, f"{PERSON_NAME}.pth"))
             print(f"☁️ Model backed up to Google Drive: {drive_voice_dir}")
        else:
             print("❌ No model file generated.")
        
    except Exception as e:
        print(f"❌ Training failed with error: {e}")
    finally:
        os.chdir(cwd_backup)

## 🎭 Step 5: Voice Conversion

Convert audio using any trained (or loaded) voice.

In [ ]:
from core.inference import VoiceConverter
from utils.registry import discover_voices

# List available voices (including those synced from Drive)
available_voices = discover_voices(models_dir="models")
print(f"Available voices: {available_voices}")

# Conversion parameters
SOURCE_AUDIO = "/content/source_audio.wav"  # Path to source audio
TARGET_VOICE = available_voices[0] if available_voices else None
OUTPUT_PATH = "/content/output_converted.wav"

if TARGET_VOICE:
    print(f"Converting audio to voice: {TARGET_VOICE}")
    registry = VoiceRegistry(models_dir="models")
    model_path = registry.get_model_path(TARGET_VOICE)
    
    if not model_path:
         # Fallback check if registry needs refresh
         registry.refresh()
         model_path = registry.get_model_path(TARGET_VOICE)

    if model_path:
        converter = VoiceConverter(model_path, device=device)
        try:
            converter.convert(SOURCE_AUDIO, OUTPUT_PATH, pitch_shift=0.0)
            print(f"✅ Conversion completed! Output saved to: {OUTPUT_PATH}")
        except Exception as e:
            print(f"❌ Conversion failed: {e}")
    else:
        print(f"❌ Could not find model for {TARGET_VOICE}")
else:
    print("❌ No trained voices available.")